In [ ]:
# default_exp player

# player
> Mathematical players for games of truth and consequences

In [ ]:
# hide
from nbdev.showdoc import *

%load_ext nb_black
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.greedy=True

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [ ]:
# export
import numpy as np
from truthiness import gym

<IPython.core.display.Javascript object>

# a run function
First we make a run function to our players. 
- An 'episode' is a full game of play. 
- Each episode uses a different random seed.
- Data for each game gets logged to csv file. 
- (The loging interface is mine but is compatible with the [tensorboard](https://www.tensorflow.org/tensorboard) logging system for deep learning)

In [ ]:
# export
def run(player, num_episode=10, env_name="ShameGame1", **env_kwargs):
    """Run a set of games"""

    # Get the env
    Env = getattr(env_name, gym)

    # Init logging

    # !
    for n in range(num_episodes):
        # Reconfig the env
        maze = create_maze(8, k=5, t=10)
        env = Env(maze=maze, **env_kwargs)

        # Reset
        x, y, Q, E = env.reset()
        done = False

        # -
        while not done:
            x, y = player(E, Q, env.moves())
            state, reward, done, _ = env.step((x, y))
            x, y, Q, E = state

        # Log data

<IPython.core.display.Javascript object>

# all players 

Here we define different simple players and dispositions to truth and consequences. 
- Honesty: $max\ E_{i,j}$
- Planned honesty: $argmax_{\pi} \sum_P E_{i,j}$
- Sensitive: $min\ Q_{i,j}$
- Planned sensitive: $argmin_{\pi} \sum_P Q_{i,j}$
- Evil: $max\ Q_{i,j}$
- Planned evil: $argmax_{\pi} \sum_P Q_{i,j}$
- Foraging: $max\ \frac{E_{i,j}}{Q_{i,j}}$
- Planned foraging: $argmax_{\pi} \sum_P \frac{E_{i,j}}{Q_{i,j}}$
- Random

We also consider a couple more complex approaches which swtich amoung the simple policies

# a random player
They play by making random moves. Simple and bad.

In [ ]:
# export
class Random:
    def __init__(self, prng=None):
        if prng is None:
            self.prng = np.random.RandomState()
        else:
            self.prng = prng

    def __call__(self, moves):
        return self.forward(moves)

    def forward(self, E, Q, moves):
        i = self.prng.randint(0, len(moves))
        move = moves[i]

        return move

<IPython.core.display.Javascript object>